In [ ]:
import pandas as pd
import rich


In [ ]:
import pandas as pd
ibm_hr = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/066_IBM_HR/sample.parquet")
tripadvisor = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/067_TripAdvisor/sample.parquet")
worldBank = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/068_WorldBank_Awards/sample.parquet")
taxonomy = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/069_Taxonomy/sample.parquet")
openfoodfacts = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/070_OpenFoodFacts/sample.parquet")
col = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/071_COL/sample.parquet")
admissions = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/072_Admissions/sample.parquet")
med_cost = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/073_Med_Cost/sample.parquet")
lift = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/074_Lift/sample.parquet")
mortality = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/075_Mortality/sample.parquet")
nba = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/076_NBA/sample.parquet")
gestational = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/077_Gestational/sample.parquet")
fires = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/078_Fires/sample.parquet")
coffee = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/079_Coffee/sample.parquet")
books = pd.read_parquet("/Users/tonypiacentini/csc-306-Project-4/src/data/080_Books/sample.parquet")


In [ ]:
data = pd.DataFrame(columns = ['DataSet','DataRaw'])
data.loc[len(data)] = ['066_IBM_HR', ibm_hr.to_csv(index=False)]
data.loc[len(data)] = ['067_TripAdvisor', tripadvisor.to_csv(index=False)]
data.loc[len(data)] = ['068_WorldBank_Awards', worldBank.to_csv(index=False)]
data.loc[len(data)] = ['069_Taxonomy', taxonomy.to_csv(index=False)]
data.loc[len(data)] = ['070_OpenFoodFacts', openfoodfacts.to_csv(index=False)]
data.loc[len(data)] = ['071_COL', col.to_csv(index=False)]
data.loc[len(data)] = ['072_Admissions', admissions.to_csv(index=False)]
data.loc[len(data)] = ['073_Med_Cost', med_cost.to_csv(index=False)]
data.loc[len(data)] = ['074_Lift', lift.to_csv(index=False)]
data.loc[len(data)] = ['075_Mortality', mortality.to_csv(index=False)]
data.loc[len(data)] = ['076_NBA', nba.to_csv(index=False)]
data.loc[len(data)] = ['077_Gestational', gestational.to_csv(index=False)]
data.loc[len(data)] = ['078_Fires', fires.to_csv(index=False)]
data.loc[len(data)] = ['079_Coffee', coffee.to_csv(index=False)]
data.loc[len(data)] = ['080_Books', books.to_csv(index=False)]

In [ ]:
avgAge = (ibm_hr['Age'].sum() / ibm_hr['Age'].count())
print(avgAge)

In [ ]:
print(data['DataRaw'])

In [ ]:
data = data.set_index('DataSet')


In [ ]:
data.head()

In [ ]:
import openai_test

In [ ]:
for name, raw in data.itertuples():
    print(name)


In [ ]:
query = "Is our average employee older than 35?"
dataframe = data.loc['066_IBM_HR']
result_df1 = openai_test.response_test(query, dataframe['DataRaw'])

In [ ]:
import ast
data_dict = ast.literal_eval(result_df1)
data_dict["question"] = query
data_dict["dataset_used"] = "066_IBM_HR"
print(data_dict)


In [ ]:
test_qa = pd.read_csv("/Users/tonypiacentini/csc-306-Project-4/src/data/test_qa.csv")  
test_qa.tail()

In [ ]:
test_qa = test_qa.set_index('dataset')

In [ ]:
import json

In [ ]:
final = pd.DataFrame(columns = ['question', 'answer', 'columns_used', "explanation", "dataset_used"])

In [ ]:
runner = 0
last_name = ""
final = []
for name, query in test_qa.itertuples():
    
    if(name != "067_TripAdvisor"):
        if (runner % 10 == 0):
            print(runner)
        dataframe = data.loc[name]
        result = openai_test.response_test(query, dataframe['DataRaw'])
        next = []
        q = []
        q.append("question")
        q.append(query)
        next.append(q)
        n = []
        n.append("dataset_used")
        n.append(name)
        next.append(n)
        next.append(result)
        final.append(next)
        runner += 1

In [ ]:
print(final[0])

In [ ]:
loaded = json.loads(final[0][2])
list_of_lists = [[key, value] for key, value in loaded.items()]
print(list_of_lists)

In [ ]:
def fix_and_parse_json(json_str):
    try:
        return json.loads(json_str)  # Try parsing directly
    except json.JSONDecodeError:
        # Fix common issues:
        json_str = json_str.rstrip(")") # Replace ) with }
        print(f"Fixed JSON: {json_str}")
        try:
            return json.loads(json_str)  # Try parsing again
        except json.JSONDecodeError as e:
            print(f"Still an error: {e}")
            return None  # Return None if still invalid


In [ ]:
def fix_and_parse_json2(json_str):
    try:
        return json.loads(json_str)  # Try parsing directly
    except json.JSONDecodeError:
        # Fix common issues:
        json_str = json_str[:-2] + "}" # Replace ) with }
        print(f"Fixed JSON: {json_str}")
        try:
            return json.loads(json_str)  # Try parsing again
        except json.JSONDecodeError as e:
            print(f"Still an error: {e}")
            return None  # Return None if still invalid

In [ ]:
fixed = []
next = []
for i in range(len(final)):
    next.append(final[i][0])
    next.append(final[i][1])
    loaded = fix_and_parse_json2(final[i][2])
    list_of_lists = [[key, value] for key, value in loaded.items()]
    for i in list_of_lists:
        next.append(i)
    next.append(list_of_lists)
    fixed.append(next)



In [ ]:
print(0)

In [ ]:
print(fixed[0])